Import the libraries.

Utility functions, to store the temporary images and to convert matrix shape.

## Dataset

## Model

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

## Training / Loading

In [ ]:
if exists(model_file):
    model.load_state_dict(torch.load(model_file))
    model.eval()
else:
    losses = []
    for epoch in range(num_epochs):
        # model.train()
        # model = model.to(device)
        train_loss = 0
        for batch_idx, data in enumerate(loader):
            img, _ = data
            img = img.view(img.size(0), -1)
            img = Variable(img)
            img = img.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(img)
            loss = loss_function(recon_batch, img, mu, logvar)
            loss.backward()
            train_loss += loss.data
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * len(img),
                    len(loader.dataset), 100. * batch_idx / len(loader),
                    loss.data / len(img)))

        losses.append(train_loss.cpu().detach().numpy())
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(loader.dataset)))
        if epoch % 10 == 0:
            save = to_img(recon_batch.cpu().data)
            save_image(save, './vae_img/image_{}.png'.format(epoch))

        torch.save(model.state_dict(), model_file)
        
    model.eval()
    # Defining the Plot Style
    plt.style.use('fivethirtyeight')
    plt.xlabel('Iterations')
    plt.ylabel('Loss')

    # Plotting the last 100 values
    plt.plot(losses)

## Display sample images

## Display latent space

In [ ]:
data_types_dict = {'label': int}
df = df.astype(data_types_dict)
print(df.dtypes)

## Testing on different space

In [ ]:
model.eval()

fig = plt.figure(figsize=(20., 5.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(2, 10),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 )


for i in range(10):
    image,_ = dataset.__getitem__(random.randint(1,37000))

    img = image.reshape(-1, 28*28)
    img = Variable(img)
    img = img.to(device) 

    reconstructed, _, _ = model(img)
    item = reconstructed.reshape(-1, 28, 28)
    item = item.cpu().detach().numpy()

    # Iterating over the grid returns the Axes.
    grid[i].imshow(image[0], cmap='gray')
    grid[i].grid(False)
    grid[10+i].imshow(item[0], cmap='gray')
    grid[10+i].grid(False)


plt.show()

In [ ]:
# Download the Fashion MNIST Dataset
fashionDataset = ...

In [ ]:
model.eval()

...